#**Pre-request**

##Mount google drive


In [1]:
### **Mount** Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


##Install pakages


In [2]:
#Install pakages
%pip install -q -r /content/drive/MyDrive/Sem-6/coding/github/fraud_detection/Extract_requirements.txt --no-cache-dir



In [3]:
project_path = "/content/drive/MyDrive/Sem-6/coding/github/fraud_detection/"
%cd $project_path
%ls /content/drive/MyDrive/Sem-6/coding/github/fraud_detection

/content/drive/MyDrive/Sem-6/coding/github/fraud_detection
clustring_requirements-lock.txt  requirements-lock.txt
clustring_requirements.txt       requirements.txt
configs/                         results/
dataset/                         run_experiment.py
Extract_requirements-lock.txt    sample_extract_requirements-lock.txt
Extract_requirements.txt         src/
notebooks/                       tests/
README.md


##Import  libs

In [4]:

import datetime
import os
import pandas as pd
import numpy as np
from scipy.stats import mode
import yaml
import logging
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler
from sklearn.manifold import TSNE
import altair as alt
from google.colab import data_table
data_table.enable_dataframe_formatter()
# Expand Colab’s table display limits
pd.set_option("display.max_columns", None)
pd.set_option("display.width", None)
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import timedelta


%pip freeze > Extract_requirements-lock.txt


#Utility Functions

##Loging

In [5]:

# Make sure results directory exists
os.makedirs("results", exist_ok=True)

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s",
    handlers=[
        logging.StreamHandler(),
        logging.FileHandler("results/data_extract.log")
    ]
)
logger = logging.getLogger(__name__)



##Config

In [6]:
def load_config(config_path="configs/baseline.yaml"):
    """Load YAML config file."""
    with open(config_path, "r") as f:
        config = yaml.safe_load(f)
    logger.info(f"✅ Loaded config from {config_path}")
    return config


## CDR dataset

In [7]:
def load_cdr(file_path, nrows=None):
    """Load a CSV file and safely parse datetime columns."""
    logger.info(f"📂 Loading file: {file_path} (nrows={nrows})")
    df = pd.read_csv(file_path, nrows=nrows)

    # Auto-detect and parse datetime columns
    for col in df.columns:
        if "datetime" in col.lower() or "time" in col.lower():
            df[col] = pd.to_datetime(df[col], errors="coerce")

    df.columns = df.columns.str.strip()  # clean header spaces
    logger.info(f"✅ Loaded {df.shape[0]} rows, {df.shape[1]} columns")
    return df


def load_all_data(config):
    """
    Load all CSVs defined in config['Agg'] into a dict of DataFrames.
    """
    agg_cfg = config["Agg"]
    base = agg_cfg["base_path"]
    files = agg_cfg["files"]

    data = {}
    for name, fname in files.items():
        path = os.path.join(base, fname)
        df = load_cdr(path)
        data[name] = df
        logger.info(f"📊 Loaded {name:<5} -> {df.shape} from {path}")
    return data

##Features

###Voice Features

In [8]:
def get_voc_feats(df, cutoff_time=None, n_events=None):
    """Extract per-user voice call features within given time window."""
    df = df.copy()
    if df.empty:
        return pd.DataFrame(columns=["phone_no_m"])

    # ✅ Time filtering (moved from build_user_snapshots)
    if cutoff_time is not None:
        df = df[df["start_datetime"] >= cutoff_time]
    if n_events:
        df = df.sort_values("start_datetime").tail(n_events)

    df["call_dur"] = pd.to_numeric(df["call_dur"], errors="coerce").fillna(0)

    df["weekday"] = pd.to_datetime(df["start_datetime"]).dt.weekday
    df["hour"] = pd.to_datetime(df["start_datetime"]).dt.hour

    feats = (
        df.groupby("phone_no_m", as_index=False)
        .agg(
            voc_total_calls=("start_datetime", "count"),
            voc_unique_contacts=("opposite_no_m", "nunique"),
            voc_total_duration=("call_dur", "sum"),
            voc_avg_duration=("call_dur", "mean"),
            voc_max_duration=("call_dur", "max"),
            voc_std_duration=("call_dur", "std"),
            voc_active_days=("weekday", "nunique"),
            voc_active_hours=("hour", "nunique"),
        )
    )
    return feats.fillna(0)


###SMS Features

In [9]:
def get_sms_feats(df, cutoff_time=None, n_events=None):
    """Extract per-user SMS features within given time window."""
    df = df.copy()
    if df.empty:
        return pd.DataFrame(columns=["phone_no_m"])

    # ✅ Time filtering (if requested)
    if cutoff_time is not None:
        df = df[df["request_datetime"] >= cutoff_time]
    if n_events:
        df = df.sort_values("request_datetime").tail(n_events)

    # ✅ Ensure calltype_id is numeric
    df["calltype_id"] = pd.to_numeric(df["calltype_id"], errors="coerce")

    # ✅ Extract hour for time-based features
    df["hour"] = pd.to_datetime(df["request_datetime"]).dt.hour

    feats = (
        df.groupby("phone_no_m", as_index=False)
        .agg(
            sms_total_msgs=("request_datetime", "count"),
            sms_unique_contacts=("opposite_no_m", "nunique"),
            sms_active_hours=("hour", "nunique"),
            # 1 → outgoing, 2 → incoming (adjust if opposite)
            sms_calltype_ratio=("calltype_id", lambda x: (x == 1).mean()),
        )
    )

    return feats.fillna(0)


###App Features

In [10]:
import pandas as pd
import datetime

def get_app_feats(df):
    """Extract per-user application usage features (aggregated monthly).
    Handles various month_id formats internally (201908, '2019-08', '2019/08', '2019-12-01').
    """
    df = df.copy()

    if df.empty:
        return pd.DataFrame(columns=[
            "phone_no_m", "app_months_active", "app_total_flow",
            "app_avg_flow", "app_std_flow",
            "app_unique_apps_mean", "app_unique_apps_max"
        ])

    # Ensure month_id exists
    if "month_id" not in df.columns:
        raise ValueError("❌ APP dataset must contain 'month_id' column.")

    # Convert flow to numeric
    df["flow"] = pd.to_numeric(df["flow"], errors="coerce").fillna(0)

    # Inline month_id parser
    def parse_month_end(x):
        if pd.isna(x):
            return pd.NaT
        s = str(x).strip()
        # Handle YYYYMM
        if s.isdigit() and len(s) == 6:
            y, m = int(s[:4]), int(s[4:])
            return pd.Timestamp(datetime.date(y, m, 1)) + pd.offsets.MonthEnd(0)
        # Handle full or partial date strings
        for fmt_try in [s, s + "-01"]:
            try:
                dt = pd.to_datetime(fmt_try, errors="coerce")
                if pd.notna(dt):
                    return dt + pd.offsets.MonthEnd(0)
            except Exception:
                continue
        return pd.NaT

    # Apply month parsing
    df["month_end"] = df["month_id"].apply(parse_month_end)
    df = df.dropna(subset=["month_end"])

    # Aggregate monthly totals
    monthly = (
        df.groupby(["phone_no_m", "month_end"])
        .agg(
            total_flow=("flow", "sum"),
            unique_apps=("busi_name", "nunique"),
        )
        .reset_index()
    )

    # Aggregate per-user statistics
    features = (
        monthly.groupby("phone_no_m")
        .agg(
            app_months_active=("month_end", "nunique"),
            app_total_flow=("total_flow", "sum"),
            app_avg_flow=("total_flow", "mean"),
            app_std_flow=("total_flow", "std"),
            app_unique_apps_mean=("unique_apps", "mean"),
            app_unique_apps_max=("unique_apps", "max"),
        )
        .reset_index()
        .fillna(0)
    )

    return features


###User Features

In [11]:
def get_user_feats(df):
    """
    Extract per-user ARPU-based features.
    Handles both YYYYMM and YYYY-MM-DD month_id formats.
    Zeros are considered inactive unless explicitly kept.
    """

    import pandas as pd
    df = df.copy()

    if "arpu_value" not in df.columns:
        raise ValueError("Expected column 'arpu_value' not found.")

    # --- Convert month_id to datetime safely ---
    def to_month_end(val):
        if pd.isna(val):
            return pd.NaT
        s = str(val).strip()
        # Handle both '2019-08-01' and '201908'
        try:
            dt = pd.to_datetime(s, errors="coerce")
            if pd.notna(dt):
                return dt + pd.offsets.MonthEnd(0)
        except Exception:
            pass
        # fallback for YYYYMM numeric
        s = s.replace("-", "").replace("/", "")
        if len(s) == 6:
            try:
                return pd.to_datetime(s + "01", format="%Y%m%d") + pd.offsets.MonthEnd(0)
            except Exception:
                return pd.NaT
        return pd.NaT

    df["month_end"] = df["month_id"].apply(to_month_end)

    # --- Convert ARPU values to numeric ---
    df["arpu_value"] = pd.to_numeric(df["arpu_value"], errors="coerce")

    # --- Filter valid ARPU entries ---
    df_valid = df[df["arpu_value"].notna() & (df["arpu_value"] > 0)]

    # --- Aggregate per user ---
    user_feats = (
        df_valid.groupby("phone_no_m", as_index=False)
        .agg(
            user_months_active=("month_end", "nunique"),
            arpu_mean=("arpu_value", "mean"),
            arpu_std=("arpu_value", "std"),
            arpu_max=("arpu_value", "max"),
            idcard_cnt=("idcard_cnt", "max"),
            label=("label", "max"),
        )
    )

    # --- If no active month found, return zero row ---
    if user_feats.empty:
        user_feats = pd.DataFrame([{
            "phone_no_m": df["phone_no_m"].iloc[0],
            "user_months_active": 0,
            "arpu_mean": 0,
            "arpu_std": 0,
            "arpu_max": 0,
            "idcard_cnt": df.get("idcard_cnt", [0])[0],
            "label": df.get("label", [0])[0],
        }])

    return user_feats


###Get feature names

In [12]:
def get_feature_names():
    ALL_FEATURE_COLUMNS = [
        # Voice
        "voc_total_calls", "voc_unique_contacts", "voc_total_duration",
        "voc_avg_duration", "voc_max_duration", "voc_std_duration",
        "voc_active_days", "voc_active_hours",
        # SMS
        "sms_total_msgs", "sms_unique_contacts", "sms_active_hours", "sms_calltype_ratio",
        # App
        "app_months_active", "app_total_flow", "app_avg_flow",
        "app_std_flow", "app_unique_apps_mean", "app_unique_apps_max",
        # User / ARPU
        "user_months_active", "arpu_mean", "arpu_std", "arpu_max",
        "idcard_cnt", "label"
    ]
    return ALL_FEATURE_COLUMNS


###App Risk Level

In [13]:
def compute_app_risk_levels(df, label_col="label", min_users=5):
    """Compute fraud risk ratio per app."""
    if df.empty or label_col not in df.columns:
        return pd.DataFrame(columns=["busi_name", "risk_level", "fraud_ratio"])

    risk_df = (
        df.groupby("busi_name", as_index=False)
        .agg(
            users=("phone_no_m", "nunique"),
            fraud_users=(label_col, "sum"),
        )
    )
    risk_df["fraud_ratio"] = risk_df["fraud_users"] / risk_df["users"]
    risk_df = risk_df[risk_df["users"] >= min_users]

    # Risk level classification
    bins = [0, 0.01, 0.05, 0.15, 0.30, 1.0]
    labels = ["Low", "Medium", "Elevated", "High", "Critical"]
    risk_df["risk_level"] = pd.cut(risk_df["fraud_ratio"], bins=bins, labels=labels)

    return risk_df

##Snapshot

###ensure dataframe

In [14]:

def ensure_dataframe(df, label, user):
    """Guarantee a DataFrame with phone_no_m even if empty or Series."""
    if df is None:
        df = pd.DataFrame()
    if isinstance(df, pd.Series):
        df = df.to_frame().T
    if not isinstance(df, pd.DataFrame):
        df = pd.DataFrame(df)
    if df.empty or "phone_no_m" not in df.columns:
        df["phone_no_m"] = [user]
    return df.reset_index(drop=True)



###Subset by window

In [15]:
def subset_by_window(df, times):
    return df[df["event_time"].isin(times)]


###Build snapshot windows

In [46]:
def build_snapshot_windows(all_user_events, event_times_sorted, user_min_time, user_max_time, window_mode, window_size,max_snapshots=1):
    """Generate time- or event-based snapshot windows for a single user."""
    snapshots_for_user = []

    if window_mode == "time":
        cutoff_time = user_min_time + datetime.timedelta(days=window_size)

        print(f"\n Cutoff is {cutoff_time} for first snapshot\n")

        while True:
            start_window = cutoff_time - datetime.timedelta(days=window_size)

            recent_events = all_user_events[
                (all_user_events["event_time"] >= start_window) &
                (all_user_events["event_time"] <= cutoff_time)
            ].copy()

            print(f"Checking {start_window} → {cutoff_time}: {len(recent_events)} events")

            if not recent_events.empty:
                snapshots_for_user.append((start_window, cutoff_time, recent_events))

            # stop when window start exceeds last event
            if start_window > user_max_time:
                break
          # ✅ Stop if we've reached max snapshots
            if max_snapshots and len(snapshots_for_user) >= max_snapshots:
                  break


            cutoff_time += datetime.timedelta(days=1)

    elif window_mode == "events":
        total_events = len(event_times_sorted)
        if total_events >= window_size:
            for start_idx in range(0, total_events - window_size + 1, 1):  # sliding by 1
                end_idx = start_idx + window_size
                recent_events = all_user_events.iloc[start_idx:end_idx].copy()
                cutoff_time = recent_events["event_time"].max()
                start_window = recent_events["event_time"].min()
                print(f"Checking events {start_idx}–{end_idx}: {len(recent_events)} records | {start_window} → {cutoff_time}")
                snapshots_for_user.append((start_window, cutoff_time, recent_events))
                    # ✅ Stop after X snapshots if user requested
                if max_snapshots and len(snapshots_for_user) >= max_snapshots:
                      break
        else:
            cutoff_time = event_times_sorted[-1]
            recent_events = all_user_events.copy()
            start_window = event_times_sorted[0]
            print(f"Checking all {len(recent_events)} records | {start_window} → {cutoff_time}")
            snapshots_for_user.append((start_window, cutoff_time, recent_events))

    else:
        raise ValueError("Invalid window_mode: must be 'time' or 'events'")

    # filter empty windows before returning
    snapshots_for_user = [s for s in snapshots_for_user if not s[2].empty]
    return snapshots_for_user


###Get event type

In [17]:
def get_event_type_for_cutoff(all_user_events, cutoff_time):
    """Return the source type of the last event before or at cutoff_time."""
    match_row = all_user_events.query("event_time <= @cutoff_time").tail(1)
    return match_row["source"].iloc[0] if not match_row.empty else "Unknown"


###Subset per source

In [18]:
def split_events_by_source(voc_user, sms_user, app_user, arpu_user, recent_events):
    """Return per-source event subsets for the given window."""
    times_in_window = set(recent_events["event_time"])
    return {
        "VOC": subset_by_window(voc_user, times_in_window),
        "SMS": subset_by_window(sms_user, times_in_window),
        "APP": subset_by_window(app_user, times_in_window),
        "ARPU": subset_by_window(arpu_user, times_in_window),
    }


###Feature extraction wrapper

In [19]:
def extract_features_for_sources(subsets, user):
    """Run feature extraction for each data source safely."""
    feats = {}
    try:
        feats["USER"] = get_user_feats(subsets["ARPU"]) if not subsets["ARPU"].empty else pd.DataFrame()
        feats["VOC"]  = get_voc_feats(subsets["VOC"])   if not subsets["VOC"].empty else pd.DataFrame()
        feats["SMS"]  = get_sms_feats(subsets["SMS"])   if not subsets["SMS"].empty else pd.DataFrame()
        feats["APP"]  = get_app_feats(subsets["APP"])   if not subsets["APP"].empty else pd.DataFrame()
    except Exception as e:
        print(f"❌ Feature extraction failed for user {user}: {e}")
        return {}

    # Ensure dataframes are valid and have phone_no_m
    for key in feats:
        feats[key] = ensure_dataframe(feats[key], key.lower(), user)
    return feats


###Combine to single snapshot row

In [61]:
def combine_features_to_snapshot(
    user, step, cutoff_time, event_type,
    window_mode, window_size, feats, all_feature_columns,start_window
):
    """Merge all per-source features into one unified snapshot row."""
    base = pd.DataFrame({
        "phone_no_m": [user],
        "snapshot_index": [step],
        "timeframe_start": [start_window],
        "timeframe_end": [cutoff_time],
        "last_event_type": [event_type],
        "window_mode": [window_mode],
        "window_size": [window_size],
    })

    snapshot = (
        base
        .merge(feats.get("VOC", pd.DataFrame()), on="phone_no_m", how="left")
        .merge(feats.get("SMS", pd.DataFrame()), on="phone_no_m", how="left")
        .merge(feats.get("APP", pd.DataFrame()), on="phone_no_m", how="left")
        .merge(feats.get("USER", pd.DataFrame()), on="phone_no_m", how="left")
    )

    snapshot = snapshot.reindex(
        columns=[
            "phone_no_m", "snapshot_index", "timeframe_start","timeframe_end",
            "last_event_type", "window_mode", "window_size"
        ] + all_feature_columns,
        fill_value=0
    )
    return snapshot


###Time-Aware User Snapshots

In [52]:

def build_user_snapshots_global(
    df_voc,
    df_sms,
    df_app_tx,
    df_arpu_tx,
    window_size=30,           # Can be days or number of events
    window_mode="time",       # "time" or "events"
    max_snapshots=None,
    max_users=None,
    save_path=None
):



    # --- Step 1: Expected feature schema ---
    ALL_FEATURE_COLUMNS = get_feature_names()

    # --- Step 2: Unified timeline ---
    df_timeline = build_unified_timeline(df_voc, df_sms, df_app_tx, df_arpu_tx)
    users = sorted(df_timeline["phone_no_m"].unique())
    total_available = len(users)
    if max_users:
        users = users[:max_users]
        print(f"⚙️ Limiting run to first {max_users} users out of {total_available} total.\n")

    snapshots = []
    # --- Step 3: Process each user ---
    for user in tqdm(users, desc="📸 Building unified snapshots"):
        user_tl = prepare_user_timeline(df_timeline, user)
        if user_tl is None:
            continue

        user_subsets = get_user_subsets(df_voc, df_sms, df_app_tx, df_arpu_tx, user)
        all_user_events, user_min_time, user_max_time = build_unified_user_events(user_subsets)

        if all_user_events is None:
            continue

        print(f"\n\n📅 User timeline range: {user_min_time} → {user_max_time}")


        # --- Step 3.2: Build rolling snapshot windows ---
        snapshots_for_user = []


        event_times_sorted = all_user_events["event_time"].tolist()

        snapshots_for_user = build_snapshot_windows(
              all_user_events,
              event_times_sorted,
              user_min_time,
              user_max_time,
              window_mode,
              window_size,
              max_snapshots=max_snapshots
          )


        if not snapshots_for_user:
            print(f"⚠️ No valid snapshots for user {user}")
            continue


        # --- Step 4: Iterate over each snapshot ---
        for step, (start_window, cutoff_time, recent_events) in enumerate(snapshots_for_user, start=1):

            # Step 1: Identify event type
            event_type = get_event_type_for_cutoff(all_user_events, cutoff_time)

            # Step 2: Split per-source subsets
            source_subsets = split_events_by_source(
              user_subsets["VOC"],
              user_subsets["SMS"],
              user_subsets["APP"],
              user_subsets["ARPU"],
              recent_events
          )


            if all(df.empty for df in source_subsets.values()):
                logger.warning(f"⚠️ Skipping empty snapshot for user {user} at {cutoff_time}")
                continue

            # Step 3: Extract features safely
            feats = extract_features_for_sources(source_subsets, user)

            # Step 4: Combine into unified snapshot row
            snapshot = combine_features_to_snapshot(
                user=user,
                step=step,
                cutoff_time=cutoff_time,
                event_type=event_type,
                window_mode=window_mode,
                window_size=window_size,
                feats=feats,
                all_feature_columns=ALL_FEATURE_COLUMNS,
                start_window=start_window
            )

            snapshots.append(snapshot)


    # --- Step 5: Combine and Save ---
    if not snapshots:
        print("⚠️ No snapshots generated — check data or window size.")
        return pd.DataFrame(columns=["phone_no_m"] + ALL_FEATURE_COLUMNS)

    tqdm.write("")   # or print("\n", flush=True)
    snapshots_df = pd.concat(snapshots, ignore_index=True).fillna(0)
    display(snapshots_df)

    return snapshots_df#save_snapshots_to_csv(snapshots_df, save_path, users)



###Build Unified Timeline

In [22]:
def build_unified_timeline(df_voc, df_sms, df_app_tx, df_arpu_tx):
    """
    Build a unified chronological timeline of all user-related events (VOC, SMS, APP, ARPU).
    Each entry = (phone_no_m, event_time, source)
    """
    import pandas as pd

    def ensure_core_fields(df, name):
        """Ensure that each dataset has 'phone_no_m', 'event_time', 'source'."""
        # Fallback for event_time naming
        if "event_time" not in df.columns:
            if "start_datetime" in df.columns:
                df["event_time"] = pd.to_datetime(df["start_datetime"], errors="coerce")
            elif "request_datetime" in df.columns:
                df["event_time"] = pd.to_datetime(df["request_datetime"], errors="coerce")
            else:
                raise ValueError(f"{name} dataset missing a valid timestamp column")

        # Fallback for source naming
        if "source" not in df.columns:
            df["source"] = name

        # Keep only relevant columns
        df = df[["phone_no_m", "event_time", "source"]].copy()
        df["event_time"] = pd.to_datetime(df["event_time"], errors="coerce")
        df = df.dropna(subset=["phone_no_m", "event_time"])
        return df

    # ✅ Normalize all datasets
    voc_tl = ensure_core_fields(df_voc, "VOC")
    sms_tl = ensure_core_fields(df_sms, "SMS")
    app_tl = ensure_core_fields(df_app_tx, "APP")
    arpu_tl = ensure_core_fields(df_arpu_tx, "ARPU")

    # ✅ Combine everything
    all_events = pd.concat([voc_tl, sms_tl, app_tl, arpu_tl], ignore_index=True)

    # ✅ Sort globally by user and timestamp
    all_events = all_events.sort_values(["phone_no_m", "event_time"]).reset_index(drop=True)

    print(f"📦 Unified timeline built with {len(all_events):,} total events "
          f"across {all_events['phone_no_m'].nunique()} users.")
    print(all_events['source'].value_counts())

    return all_events


###Prepare user timeline

In [23]:
def prepare_user_timeline(df_timeline, user):
    """
    Extract and clean a user's timeline data (event_time sorted).
    Returns: cleaned DataFrame or None if empty.
    """
    user_tl = df_timeline.query("phone_no_m == @user").copy()
    if user_tl.empty:
        return None

    user_tl["event_time"] = pd.to_datetime(user_tl["event_time"], errors="coerce")
    user_tl = user_tl.dropna(subset=["event_time"]).sort_values("event_time")
    return user_tl if not user_tl.empty else None


###Get user subset data

In [24]:
def get_user_subsets(df_voc, df_sms, df_app_tx, df_arpu_tx, user):
    """
    Return per-source subsets (VOC, SMS, APP, ARPU) for the given user.
    """
    return {
        "VOC": df_voc.query("phone_no_m == @user").copy(),
        "SMS": df_sms.query("phone_no_m == @user").copy(),
        "APP": df_app_tx.query("phone_no_m == @user").copy(),
        "ARPU": df_arpu_tx.query("phone_no_m == @user").copy(),
    }


###Build unified user events

In [25]:
def build_unified_user_events(user_subsets):
    """
    Merge all per-source DataFrames into a single unified chronological event list.
    Cleans timestamps and removes NaNs.
    Returns: (DataFrame, min_time, max_time)
    """
    all_user_events = pd.concat(user_subsets.values(), ignore_index=True)
    all_user_events["event_time"] = pd.to_datetime(all_user_events["event_time"], errors="coerce")
    all_user_events = all_user_events.dropna(subset=["event_time"]).sort_values("event_time")

    if all_user_events.empty:
        return None, None, None

    event_times_sorted = all_user_events["event_time"].tolist()
    return all_user_events, event_times_sorted[0], event_times_sorted[-1]


#Excute pipeline

##Config and summary

In [26]:
# 1️⃣ Load config and all datasets
config = load_config("/content/drive/MyDrive/Sem-6/coding/github/fraud_detection/configs/baseline.yaml")
data = load_all_data(config)


# 2️⃣ Extract individual datasets from the returned dictionary
df_voc = data["voc"]
df_sms = data["sms"]
df_app = data["app"]
df_user = data["user"]


# Voice
df_voc["source"] = "VOC"
df_voc["event_time"] = pd.to_datetime(df_voc.get("start_datetime", df_voc.get("event_time")), errors="coerce")

# SMS
df_sms["source"] = "SMS"
df_sms["event_time"] = pd.to_datetime(df_sms.get("request_datetime", df_sms.get("event_time")), errors="coerce")

# App
df_app["source"] = "APP"
df_app["event_time"] = pd.to_datetime(df_app["event_time"], errors="coerce")

# ARPU (User)
df_user["source"] = "ARPU"
df_user["event_time"] = pd.to_datetime(df_user["event_time"], errors="coerce")

print("✅ All datasets standardized and ready for timeline merge:")
print(f"  VOC  → {len(df_voc):,} records")
print(f"  SMS  → {len(df_sms):,} records")
print(f"  APP  → {len(df_app):,} records")
print(f"  ARPU → {len(df_user):,} records")


df_timeline = build_unified_timeline(df_voc, df_sms, df_app, df_user)

# (Optional) Inspect one random user timeline
some_user = df_timeline["phone_no_m"].dropna().sample(1).iloc[0]

print(f"🔍 Inspecting timeline for user: {some_user}")
user_timeline = df_timeline[df_timeline["phone_no_m"] == some_user]

print(f"Total events for this user: {len(user_timeline)}")
print(user_timeline["source"].value_counts())


✅ All datasets standardized and ready for timeline merge:
  VOC  → 48,190 records
  SMS  → 64,225 records
  APP  → 26,142 records
  ARPU → 365 records
📦 Unified timeline built with 138,922 total events across 60 users.
source
SMS     64225
VOC     48190
APP     26142
ARPU      365
Name: count, dtype: int64
🔍 Inspecting timeline for user: 5be5a5901aa0e5701f3e6bae83c479c127c48d3b3678084052130f65c38b29a4effe50d2081ed51acc2acf6768005879cc8421086456ae92d8b0aba0fcd6319a
Total events for this user: 4106
source
VOC     1885
SMS     1362
APP      851
ARPU       8
Name: count, dtype: int64


##Genrate snapshot

In [1]:

save_path = config["Agg"]["save_path"]
    # Filter invalid ARPU rows

for df, label in [(df_user, "VOC"), (df_sms, "SMS"), (df_app, "APP"), (df_user, "ARPU")]:
            # Normalize event_time field
            if "event_time" not in df.columns or df["event_time"].isna().all():
                for alt_col in ["start_datetime", "request_datetime", "date", "busi_date"]:
                    if alt_col in df.columns:
                        df["event_time"] = pd.to_datetime(df[alt_col], errors="coerce")
                        break

snapshots_df = build_user_snapshots_global(
    df_voc=df_voc,
    df_sms=df_sms,
    df_app_tx=df_app,
    df_arpu_tx=df_user,
    window_size=1,
    window_mode="time",
    max_users=100,
    max_snapshots=1

)

# Save output snapshot
timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
os.makedirs(save_path, exist_ok=True)

output_file = os.path.join(save_path, f"user_snapshots_{timestamp}.csv")
snapshots_df.to_csv(output_file, index=False)

logger.info(f"✅ Feature extraction completed and saved to: {output_file}")
print(f"✅ Snapshot file saved to: {output_file}")


NameError: name 'config' is not defined